# 001 - CTE

```SQL
DROP TABLE IF EXISTS ventas_caoticas;

-- Creación de la tabla "Pesadilla de Datos"
CREATE TABLE ventas_caoticas (
    id_transaccion VARCHAR(50),
    fecha_venta VARCHAR(50), -- Formato inconsistente (Error 4)
    producto VARCHAR(100),    -- Espacios y mayúsculas (Error 2 y 6)
    categoria VARCHAR(50),   -- Typos y nulos (Error 1 y 10)
    monto_total FLOAT         -- Outliers y caracteres (Error 8 y 9)
);

-- Inserción de basura real
INSERT INTO ventas_caoticas (id_transaccion, fecha_venta, producto, categoria, monto_total)
VALUES
    ('101', '2023-10-01', ' Espresso', 'Bebida', 3.50),      -- Espacio inicial
    ('102', '01/10/2023', 'ESPRESSO', 'bebida', 3.50),       -- Formato fecha y minúsculas
    ('102', '01/10/2023', 'ESPRESSO', 'bebida', 3.50),       -- DUPLICADO EXACTO (Error 3)
    ('103', '2023-10-02', 'Latte ', 'BEBIDA', 4500.00),     -- OUTLIER/VALOR EXTREMO (Error 9)
    ('104', '2023-10-02', 'Muffin ', NULL, 2.50),            -- VALOR NULO (Error 1)
    ('105', '2023-10-03', ' Galleta', 'Comidat', 2.00),      -- TYPO EN CATEGORÍA (Error 10)
    ('106', '2023-10-03', 'galleta', 'Comida', 2.00),        -- Mismo producto, distinta grafía
    ('107', '2023-10-04', 'Espresso', 'Bebida', -10.00),     -- MONTO NEGATIVO SIN SENTIDO (Error 8)
    ('108', '2023.10.05', 'Latte', 'Bebida', 4.50);          -- Otro formato de fecha

SELECT * FROM ventas_caoticas;
```

In [1]:
import pandas as pd
import numpy as np

In [3]:
df_vc = pd.read_csv('DATA/001/ventas_caoticas.csv')

df_vc.head()

,id_transaccion,fecha_venta,producto,categoria,monto_total
0,101,2023-10-01,Espresso,Bebida,3.5
1,102,01/10/2023,ESPRESSO,bebida,3.5
2,102,01/10/2023,ESPRESSO,bebida,3.5
3,103,2023-10-02,Latte,BEBIDA,4500.0
4,104,2023-10-02,Muffin,NaN,2.5


In [7]:
# 1. Limpiar Nulos y Texto (Errores 1, 2, 6 y 10)
df_vc['producto'] = df_vc['producto'].str.strip().str.upper()
df_vc['categoria'] = df_vc['categoria'].str.strip().str.upper().fillna('SIN CATEGORÍA')

# 2. Corregir typos específicos (Error 10)
df_vc['categoria'] = df_vc['categoria'].replace('COMIDAT','COMIDA')

# 3. Eliminar duplicados (Error 3)
df_vc = df_vc.drop_duplicates()

# 4. Manejar Outliers/Valores extremos (Error 8 y 9)
df_vc = df_vc[(df_vc['monto_total'] > 0) & (df_vc['monto_total'] < 100)]

# 5. Agrupar
res = df_vc.groupby(['categoria','producto'])['monto_total'].sum().reset_index()

res

,categoria,producto,monto_total
0,BEBIDA,ESPRESSO,7.0
1,BEBIDA,LATTE,4.5
2,COMIDA,GALLETA,4.0
3,SIN CATEGORÍA,MUFFIN,2.5


```SQL
WITH paso_1_duplicasdos AS (
    SELECT DISTINCT * FROM ventas_caoticas
),
paso_2_limpieza_texto AS (
    SELECT
        id_transaccion,
        UPPER(TRIM(producto)) AS producto_limpio,
        COALESCE(UPPER(TRIM(categoria)), 'SIN CATEGORIA') AS categoria_limpia,
        monto_total
    FROM paso_1_duplicasdos
),
paso_3_filtros AS(
    SELECT * FROM paso_2_limpieza_texto
    WHERE monto_total > 0 AND monto_total < 100
)
SELECT
    categoria_limpia,
    producto_limpio,
    SUM(monto_total) AS total_real
FROM paso_3_filtros
GROUP BY 1, 2;
```